In [10]:
import pandas as pd

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
#먼저 데이터 읽어오기
df=pd.read_excel('../optimal_data2/Continous_2weeks_28day_8term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

In [20]:
#스케일링 먼저 하고 ndarray를 dataframe으로 변환
scaler = StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)

Result=[[0 for j in range(4)] for i in range(10)]
Count= int(322/10)*8
for i in range(10):
    #마지막 그룹은 34명
    if i==9:
        X_test=X.iloc[Count*i:-1]
        X_train=X.drop(X.index[Count*i:-1])
        y_test=y.iloc[Count*i:-1]
        y_train=y.drop(y.index[Count*i:-1])
    
    #모든 그룹은 32명씩gi
    X_test=X.iloc[Count*i:Count*(i+1)]
    X_train=X.drop(X.index[Count*i:Count*(i+1)])
    y_test=y.iloc[Count*i:Count*(i+1)]
    y_train=y.drop(y.index[Count*i:Count*(i+1)])
    #모델 정의, 예측
    model = svm.SVC(kernel='rbf') # rbf Kernel
    model.fit(X_train, y_train)
    
    predict=model.predict(X_test)
        
    #Accuracy
    print("[{}]Accuracy : {}".format(i,model.score(X_test,y_test)))

    #f1score
    f1 = f1_score(y_test,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    
    #precision/recall
    list=sk(y_test,predict,average='weighted')
    print("[{}]Precision : {}".format(i,list[0]))
    print("[{}]Recall : {}".format(i,list[1]))
    

    #결과 배열에 넣기
    Result[i][0]=model.score(X_test,y_test)
    Result[i][1]=f1
    Result[i][2]=list[0]
    Result[i][3]=list[1]

[0]Accuracy : 0.6484375
[0]F1score : 0.581636402027027
[0]Precision : 0.529410677314404
[0]Recall : 0.6484375
[1]Accuracy : 0.66796875
[1]F1score : 0.5801317294159043
[1]Precision : 0.5956101190476191
[1]Recall : 0.66796875


C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2]Accuracy : 0.70703125
[2]F1score : 0.6555741864130022
[2]Precision : 0.637721706081081
[2]Recall : 0.70703125
[3]Accuracy : 0.71875
[3]F1score : 0.6616615853658536
[3]Precision : 0.6226839048672566
[3]Recall : 0.71875


C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[4]Accuracy : 0.53515625
[4]F1score : 0.46374708095485206
[4]Precision : 0.44112847222222223
[4]Recall : 0.53515625
[5]Accuracy : 0.74609375
[5]F1score : 0.6832141500454101
[5]Precision : 0.6673301021836506
[5]Recall : 0.74609375
[6]Accuracy : 0.65234375
[6]F1score : 0.6100986949651265
[6]Precision : 0.5732083526183549
[6]Recall : 0.65234375
[7]Accuracy : 0.609375
[7]F1score : 0.529835417527904
[7]Precision : 0.5592540322580646
[7]Recall : 0.609375


C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[8]Accuracy : 0.6875
[8]F1score : 0.6286567207073971
[8]Precision : 0.5911462447340706
[8]Recall : 0.6875


C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[9]Accuracy : 0.6796875
[9]F1score : 0.6608651534822736
[9]Precision : 0.6430582524271844
[9]Recall : 0.6796875


C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
Result

[[0.6484375, 0.581636402027027, 0.529410677314404, 0.6484375],
 [0.66796875, 0.5801317294159043, 0.5956101190476191, 0.66796875],
 [0.70703125, 0.6555741864130022, 0.637721706081081, 0.70703125],
 [0.71875, 0.6616615853658536, 0.6226839048672566, 0.71875],
 [0.53515625, 0.46374708095485206, 0.44112847222222223, 0.53515625],
 [0.74609375, 0.6832141500454101, 0.6673301021836506, 0.74609375],
 [0.65234375, 0.6100986949651265, 0.5732083526183549, 0.65234375],
 [0.609375, 0.529835417527904, 0.5592540322580646, 0.609375],
 [0.6875, 0.6286567207073971, 0.5911462447340706, 0.6875],
 [0.6796875, 0.6608651534822736, 0.6430582524271844, 0.6796875]]